#### Data format cleaning
Data in a file is stored not in a proper way - time to change it!

In [225]:
from pathlib import Path
import pandas as pd

In [226]:
# Paths
csv_path = Path() / "csv"
plots_path = Path() / "plots"

In [227]:
df = pd.read_csv(csv_path / 'lego_to_be_cleaned.csv')

In [228]:
df.head(2)

,Number,Name,Theme,Subtheme,Year,Pieces,Minifigs,Availability,Retired,ReleasedDate,...,Retail,Paid,Value,Growth,Condition,Date,Notes,Collection,Status,URL
0,75144-1,Snowspeeder,Star Wars,Ultimate Collector Series,2017,1703,2,Exclusive,True,05/05/2017,...,199;99 €,199;99 €,330;40 €,65.21,New,NaN,NaN,Default,Owned,https://www.brickeconomy.com/set/75144-1/lego-...
1,10251-1,Brick Bank,Icons,Modular Buildings,2016,2380,6,RetailLimited,True,02/01/2016,...,149;99 €,149;99 €,494;81 €,229.89,New,NaN,NaN,Default,Owned,https://www.brickeconomy.com/set/10251-1/lego-...


In [229]:
# Delete all not wanted columns
df = df.drop(['Subtheme', 'Paid', 'Growth', 'Condition', 'Date', 'Notes', 'Collection', 'Status', 'URL'], axis='columns')

In [230]:
# All column names should start with a lowerletter and have '_' where space is required
df.columns = [col.lower() for col in df.columns]
df.rename(columns={'releaseddate': 'released_date'}, inplace=True)
df.rename(columns={'retireddate': 'retired_date'}, inplace=True)

df.head()

,number,name,theme,year,pieces,minifigs,availability,retired,released_date,retired_date,retail,value
0,75144-1,Snowspeeder,Star Wars,2017,1703,2,Exclusive,True,05/05/2017,15/01/2019,199;99 €,330;40 €
1,10251-1,Brick Bank,Icons,2016,2380,6,RetailLimited,True,02/01/2016,14/11/2018,149;99 €,494;81 €
2,10252-1,Volkswagen Beetle,Icons,2016,1167,0,Exclusive,True,01/08/2016,03/12/2020,89;99 €,113;14 €
3,10253-1,Big Ben,Icons,2016,4163,0,Exclusive,True,02/07/2016,25/11/2018,219;99 €,327;12 €
4,10702-1,Creative Building Set,Classic,2016,583,0,Retail,True,02/01/2016,24/11/2017,24;99 €,30;14 €


In [231]:
# Checking for ‘NaN’ - empty cells in columns
print(df.isnull().sum())

# Counting the number of rows with missing values
num_rows_with_nan = df[df.isnull().any(axis=1)].shape[0]
print(f'Number of rows with at least one NaN value: {num_rows_with_nan}.')

number            0
name              1
theme             0
year              0
pieces            0
minifigs          0
availability      0
retired           0
released_date    19
retired_date     15
retail            0
value             0
dtype: int64
Number of rows with at least one NaN value: 35.


In [232]:
# Removing rows with at least one NaN value
df = df.dropna()
print(df.isnull().sum())

number           0
name             0
theme            0
year             0
pieces           0
minifigs         0
availability     0
retired          0
released_date    0
retired_date     0
retail           0
value            0
dtype: int64


In [233]:
# Function to clean and convert currency values
def clean_currency(value):
    try:
        # Remove the currency symbol and replace ';' with '.'
        value = value.replace('€', '').replace(';', '.').strip()
        # Convert to float
        return float(value)
    except ValueError:
        # Return None if conversion is not possible
        return None

# Apply the function to the 'retail' and 'value' columns
df['retail'] = df['retail'].apply(clean_currency)
df['value'] = df['value'].apply(clean_currency)

# Drop rows where conversion to float was not possible
df = df.dropna(subset=['retail', 'value']).reset_index(drop=True)

df.head()

,number,name,theme,year,pieces,minifigs,availability,retired,released_date,retired_date,retail,value
0,75144-1,Snowspeeder,Star Wars,2017,1703,2,Exclusive,True,05/05/2017,15/01/2019,199.99,330.40
1,10251-1,Brick Bank,Icons,2016,2380,6,RetailLimited,True,02/01/2016,14/11/2018,149.99,494.81
2,10252-1,Volkswagen Beetle,Icons,2016,1167,0,Exclusive,True,01/08/2016,03/12/2020,89.99,113.14
3,10253-1,Big Ben,Icons,2016,4163,0,Exclusive,True,02/07/2016,25/11/2018,219.99,327.12
4,10702-1,Creative Building Set,Classic,2016,583,0,Retail,True,02/01/2016,24/11/2017,24.99,30.14


In [234]:
# Function to convert date format and handle invalid dates
def convert_date_format(date_str):
    try:
        return pd.to_datetime(date_str, format="%d/%m/%Y")
    except ValueError:
        return pd.NaT

# Apply the function to the date columns using .loc to avoid the SettingWithCopyWarning
df.loc[:, 'released_date'] = df['released_date'].apply(convert_date_format)
df.loc[:, 'retired_date'] = df['retired_date'].apply(convert_date_format)

# Remove rows with invalid dates
df = df.dropna(subset=['released_date', 'retired_date']).reset_index(drop=True)

df.head()

,number,name,theme,year,pieces,minifigs,availability,retired,released_date,retired_date,retail,value
0,75144-1,Snowspeeder,Star Wars,2017,1703,2,Exclusive,True,2017-05-05 00:00:00,2019-01-15 00:00:00,199.99,330.40
1,10251-1,Brick Bank,Icons,2016,2380,6,RetailLimited,True,2016-01-02 00:00:00,2018-11-14 00:00:00,149.99,494.81
2,10252-1,Volkswagen Beetle,Icons,2016,1167,0,Exclusive,True,2016-08-01 00:00:00,2020-12-03 00:00:00,89.99,113.14
3,10253-1,Big Ben,Icons,2016,4163,0,Exclusive,True,2016-07-02 00:00:00,2018-11-25 00:00:00,219.99,327.12
4,10702-1,Creative Building Set,Classic,2016,583,0,Retail,True,2016-01-02 00:00:00,2017-11-24 00:00:00,24.99,30.14


In [235]:
# Checking for duplicates in the data
duplicates_num = df.duplicated().sum()

if duplicates_num == 0:
    print('There are no duplicates in the data.')
else:
    print(f'There are {duplicates_num} duplicates in the data.')
    duplicates = df[df.duplicated(keep=False)]
    print('Duplicates:')
    print(duplicates)
    
    # Removing duplicates
    df = df.drop_duplicates()
    print(f'Duplicates have been removed. The cleaned data now has {df.shape[0]} rows.')


There are 5 duplicates in the data.
Duplicates:
       number              name         theme  year  pieces  minifigs  \
8817   8748-1  Rhotuka Spinners      Bionicle  2005       5         0   
8818   8748-1  Rhotuka Spinners      Bionicle  2005       5         0   
11407   210-2   Small Store Set  System i Leg  1958      30         0   
11408   210-2   Small Store Set  System i Leg  1958      30         0   
11420   226-1     8 Named Beams  System i Leg  1958       8         0   
11421   226-1     8 Named Beams  System i Leg  1958       8         0   
11439   259-1    1:87 VW Pickup  System i Leg  1958       0         0   
11440   259-1    1:87 VW Pickup  System i Leg  1958       0         0   
11446   308-3      Fire Station  System i Leg  1958     111         0   
11447   308-3      Fire Station  System i Leg  1958     111         0   

      availability  retired        released_date         retired_date  retail  \
8817        Retail     True  2005-01-01 00:00:00  2007-01-01 00:00:

In [236]:
# Checking for duplicates in the data
duplicates_num = df.duplicated().sum()
duplicates_num

0

In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17304 entries, 0 to 17308
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   number         17304 non-null  object 
 1   name           17304 non-null  object 
 2   theme          17304 non-null  object 
 3   year           17304 non-null  int64  
 4   pieces         17304 non-null  int64  
 5   minifigs       17304 non-null  int64  
 6   availability   17304 non-null  object 
 7   retired        17304 non-null  bool   
 8   released_date  17304 non-null  object 
 9   retired_date   17304 non-null  object 
 10  retail         17304 non-null  float64
 11  value          17304 non-null  float64
dtypes: bool(1), float64(2), int64(3), object(6)
memory usage: 1.6+ MB


In [238]:
df.reset_index(drop=True)

,number,name,theme,year,pieces,minifigs,availability,retired,released_date,retired_date,retail,value
0,75144-1,Snowspeeder,Star Wars,2017,1703,2,Exclusive,True,2017-05-05 00:00:00,2019-01-15 00:00:00,199.99,330.40
1,10251-1,Brick Bank,Icons,2016,2380,6,RetailLimited,True,2016-01-02 00:00:00,2018-11-14 00:00:00,149.99,494.81
2,10252-1,Volkswagen Beetle,Icons,2016,1167,0,Exclusive,True,2016-08-01 00:00:00,2020-12-03 00:00:00,89.99,113.14
3,10253-1,Big Ben,Icons,2016,4163,0,Exclusive,True,2016-07-02 00:00:00,2018-11-25 00:00:00,219.99,327.12
4,10702-1,Creative Building Set,Classic,2016,583,0,Retail,True,2016-01-02 00:00:00,2017-11-24 00:00:00,24.99,30.14
...,...,...,...,...,...,...,...,...,...,...,...,...
17299,662403-1,Zombie with Burning Baby Zombie and TNT,Minecraft,2024,12,2,Promotional,True,2024-01-01 00:00:00,2024-02-01 00:00:00,0.00,4.73
17300,9790-1,ROBOLAB Team Challenge Set,Education,1999,725,0,Retail,True,1999-01-01 00:00:00,2001-01-01 00:00:00,175.99,193.78
17301,2000443-1,Workshop Kit Freewheeler,Education,2015,82,1,RetailLimited,True,2015-01-01 00:00:00,2017-01-01 00:00:00,8.79,11.00
17302,2000442-1,Workshop Kit Spinning Top,Education,2015,10,0,Retail,True,2015-01-01 00:00:00,2017-01-01 00:00:00,8.79,23.77


In [239]:
df.to_csv(csv_path / 'lego.csv')

In [240]:
df

,number,name,theme,year,pieces,minifigs,availability,retired,released_date,retired_date,retail,value
0,75144-1,Snowspeeder,Star Wars,2017,1703,2,Exclusive,True,2017-05-05 00:00:00,2019-01-15 00:00:00,199.99,330.40
1,10251-1,Brick Bank,Icons,2016,2380,6,RetailLimited,True,2016-01-02 00:00:00,2018-11-14 00:00:00,149.99,494.81
2,10252-1,Volkswagen Beetle,Icons,2016,1167,0,Exclusive,True,2016-08-01 00:00:00,2020-12-03 00:00:00,89.99,113.14
3,10253-1,Big Ben,Icons,2016,4163,0,Exclusive,True,2016-07-02 00:00:00,2018-11-25 00:00:00,219.99,327.12
4,10702-1,Creative Building Set,Classic,2016,583,0,Retail,True,2016-01-02 00:00:00,2017-11-24 00:00:00,24.99,30.14
...,...,...,...,...,...,...,...,...,...,...,...,...
17304,662403-1,Zombie with Burning Baby Zombie and TNT,Minecraft,2024,12,2,Promotional,True,2024-01-01 00:00:00,2024-02-01 00:00:00,0.00,4.73
17305,9790-1,ROBOLAB Team Challenge Set,Education,1999,725,0,Retail,True,1999-01-01 00:00:00,2001-01-01 00:00:00,175.99,193.78
17306,2000443-1,Workshop Kit Freewheeler,Education,2015,82,1,RetailLimited,True,2015-01-01 00:00:00,2017-01-01 00:00:00,8.79,11.00
17307,2000442-1,Workshop Kit Spinning Top,Education,2015,10,0,Retail,True,2015-01-01 00:00:00,2017-01-01 00:00:00,8.79,23.77
